In [1]:
import time
import random
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [2]:
df=pd.read_excel("Alzheimer.xlsx")

In [3]:
df

,Group,M/F,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF
0,Nondemented,M,87,14,2.0,27.0,0,1987,696,883
1,Nondemented,M,88,14,2.0,30.0,0,2004,681,876
2,Demented,M,75,12,NaN,23.0,5,1678,736,1046
3,Demented,M,76,12,NaN,28.0,5,1738,713,101
4,Demented,M,80,12,NaN,22.0,5,1698,701,1034
...,...,...,...,...,...,...,...,...,...,...
368,Demented,M,82,16,1.0,28.0,5,1693,694,1037
369,Demented,M,86,16,1.0,26.0,5,1688,675,104
370,Nondemented,F,61,13,2.0,30.0,0,1319,801,1331
371,Nondemented,F,63,13,2.0,30.0,0,1327,796,1323


In [4]:
df.describe()

,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF
count,373.000000,373.000000,354.000000,371.000000,373.000000,373.000000,373.000000,373.000000
mean,77.013405,14.597855,2.460452,27.342318,1.774799,1488.128686,662.611260,1098.439678
std,7.640957,2.876339,1.134005,3.683244,2.291337,176.139286,203.835828,341.268372
min,60.000000,6.000000,1.000000,4.000000,0.000000,1106.000000,7.000000,1.000000
25%,71.000000,12.000000,2.000000,27.000000,0.000000,1357.000000,694.000000,1054.000000
50%,77.000000,15.000000,2.000000,29.000000,0.000000,1470.000000,724.000000,1183.000000
75%,82.000000,16.000000,3.000000,30.000000,5.000000,1597.000000,751.000000,1291.000000
max,98.000000,23.000000,5.000000,30.000000,5.000000,2004.000000,837.000000,1587.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373 entries, 0 to 372
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Group   373 non-null    object 
 1   M/F     373 non-null    object 
 2   Age     373 non-null    int64  
 3   EDUC    373 non-null    int64  
 4   SES     354 non-null    float64
 5   MMSE    371 non-null    float64
 6   CDR     373 non-null    int64  
 7   eTIV    373 non-null    int64  
 8   nWBV    373 non-null    int64  
 9   ASF     373 non-null    int64  
dtypes: float64(2), int64(6), object(2)
memory usage: 29.3+ KB


In [6]:
df.isnull().sum()

Group     0
M/F       0
Age       0
EDUC      0
SES      19
MMSE      2
CDR       0
eTIV      0
nWBV      0
ASF       0
dtype: int64

In [7]:
from sklearn.impute import SimpleImputer

In [8]:
imputer=SimpleImputer(missing_values=np.nan, strategy="mean")

In [9]:
eksik_veriler=df.iloc[:,2:10].values

In [10]:
eksik_veriler

array([[8.700e+01, 1.400e+01, 2.000e+00, ..., 1.987e+03, 6.960e+02,
        8.830e+02],
       [8.800e+01, 1.400e+01, 2.000e+00, ..., 2.004e+03, 6.810e+02,
        8.760e+02],
       [7.500e+01, 1.200e+01,       nan, ..., 1.678e+03, 7.360e+02,
        1.046e+03],
       ...,
       [6.100e+01, 1.300e+01, 2.000e+00, ..., 1.319e+03, 8.010e+02,
        1.331e+03],
       [6.300e+01, 1.300e+01, 2.000e+00, ..., 1.327e+03, 7.960e+02,
        1.323e+03],
       [6.500e+01, 1.300e+01, 2.000e+00, ..., 1.333e+03, 8.010e+02,
        1.317e+03]])

In [11]:
imputer=imputer.fit(eksik_veriler)

In [12]:
eksik_veriler=imputer.transform(eksik_veriler)

In [13]:
eksik_veriler

array([[8.70000000e+01, 1.40000000e+01, 2.00000000e+00, ...,
        1.98700000e+03, 6.96000000e+02, 8.83000000e+02],
       [8.80000000e+01, 1.40000000e+01, 2.00000000e+00, ...,
        2.00400000e+03, 6.81000000e+02, 8.76000000e+02],
       [7.50000000e+01, 1.20000000e+01, 2.46045198e+00, ...,
        1.67800000e+03, 7.36000000e+02, 1.04600000e+03],
       ...,
       [6.10000000e+01, 1.30000000e+01, 2.00000000e+00, ...,
        1.31900000e+03, 8.01000000e+02, 1.33100000e+03],
       [6.30000000e+01, 1.30000000e+01, 2.00000000e+00, ...,
        1.32700000e+03, 7.96000000e+02, 1.32300000e+03],
       [6.50000000e+01, 1.30000000e+01, 2.00000000e+00, ...,
        1.33300000e+03, 8.01000000e+02, 1.31700000e+03]])

In [14]:
# encoding group

In [15]:
type(df["Group"])

pandas.core.series.Series

In [16]:
ham_veri=pd.DataFrame(df["Group"],columns=["Group"])
ham_veri

,Group
0,Nondemented
1,Nondemented
2,Demented
3,Demented
4,Demented
...,...
368,Demented
369,Demented
370,Nondemented
371,Nondemented


In [17]:
# sütun tipini gör -> object
ham_veri.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373 entries, 0 to 372
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Group   373 non-null    object
dtypes: object(1)
memory usage: 3.0+ KB


In [18]:
# sütun tipini "category" ye dönüştür
ham_veri["Group"]=ham_veri["Group"].astype("category")

In [19]:
# sütun tipini tekrar gör -> object
ham_veri.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373 entries, 0 to 372
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   Group   373 non-null    category
dtypes: category(1)
memory usage: 637.0 bytes


In [20]:
# kategorik numerik veriler ata ve yeni bir sütun yarat
ham_veri["Group"]=ham_veri["Group"].cat.codes
ham_veri

,Group
0,2
1,2
2,1
3,1
4,1
...,...
368,1
369,1
370,2
371,2


In [21]:
# encoding m/f

In [22]:
type(df["M/F"])

pandas.core.series.Series

In [23]:
ham_veri_mf=pd.DataFrame(df["M/F"],columns=["M/F"])
ham_veri_mf

,M/F
0,M
1,M
2,M
3,M
4,M
...,...
368,M
369,M
370,F
371,F


In [24]:
# sütun tipini gör -> object
ham_veri_mf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373 entries, 0 to 372
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   M/F     373 non-null    object
dtypes: object(1)
memory usage: 3.0+ KB


In [25]:
# sütun tipini "category" ye dönüştür
ham_veri_mf["M/F"]=ham_veri_mf["M/F"].astype("category")

In [26]:
# sütun tipini tekrar gör -> object
ham_veri_mf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373 entries, 0 to 372
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   M/F     373 non-null    category
dtypes: category(1)
memory usage: 629.0 bytes


In [27]:
# kategorik numerik veriler ata ve yeni bir sütun yarat
ham_veri_mf["M/F"]=ham_veri_mf["M/F"].cat.codes
ham_veri_mf

,M/F
0,1
1,1
2,1
3,1
4,1
...,...
368,1
369,1
370,0
371,0


In [28]:
# dataframe birleştirme işlemi
veri=pd.concat([ham_veri,ham_veri_mf],axis=1)

In [29]:
veri

,Group,M/F
0,2,1
1,2,1
2,1,1
3,1,1
4,1,1
...,...,...
368,1,1
369,1,1
370,2,0
371,2,0


In [30]:
sonuc=pd.DataFrame(data=eksik_veriler,index=range(373),columns=["Age","EDUC","SES","MMSE","CDR","eTIV","nWBV","ASF"])

In [31]:
sonuc

,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF
0,87.0,14.0,2.000000,27.0,0.0,1987.0,696.0,883.0
1,88.0,14.0,2.000000,30.0,0.0,2004.0,681.0,876.0
2,75.0,12.0,2.460452,23.0,5.0,1678.0,736.0,1046.0
3,76.0,12.0,2.460452,28.0,5.0,1738.0,713.0,101.0
4,80.0,12.0,2.460452,22.0,5.0,1698.0,701.0,1034.0
...,...,...,...,...,...,...,...,...
368,82.0,16.0,1.000000,28.0,5.0,1693.0,694.0,1037.0
369,86.0,16.0,1.000000,26.0,5.0,1688.0,675.0,104.0
370,61.0,13.0,2.000000,30.0,0.0,1319.0,801.0,1331.0
371,63.0,13.0,2.000000,30.0,0.0,1327.0,796.0,1323.0


In [32]:
yeni_veri=pd.concat([veri,sonuc],axis=1)

In [33]:
yeni_veri

,Group,M/F,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF
0,2,1,87.0,14.0,2.000000,27.0,0.0,1987.0,696.0,883.0
1,2,1,88.0,14.0,2.000000,30.0,0.0,2004.0,681.0,876.0
2,1,1,75.0,12.0,2.460452,23.0,5.0,1678.0,736.0,1046.0
3,1,1,76.0,12.0,2.460452,28.0,5.0,1738.0,713.0,101.0
4,1,1,80.0,12.0,2.460452,22.0,5.0,1698.0,701.0,1034.0
...,...,...,...,...,...,...,...,...,...,...
368,1,1,82.0,16.0,1.000000,28.0,5.0,1693.0,694.0,1037.0
369,1,1,86.0,16.0,1.000000,26.0,5.0,1688.0,675.0,104.0
370,2,0,61.0,13.0,2.000000,30.0,0.0,1319.0,801.0,1331.0
371,2,0,63.0,13.0,2.000000,30.0,0.0,1327.0,796.0,1323.0


In [34]:
yeni_veri.isnull().sum()

Group    0
M/F      0
Age      0
EDUC     0
SES      0
MMSE     0
CDR      0
eTIV     0
nWBV     0
ASF      0
dtype: int64

In [35]:
yeni_veri.shape

(373, 10)

In [36]:
# sütun adları -> train

col_names =yeni_veri.columns

col_names

Index(['Group', 'M/F', 'Age', 'EDUC', 'SES', 'MMSE', 'CDR', 'eTIV', 'nWBV',
       'ASF'],
      dtype='object')

In [37]:
# input - output ayrımı 
y=yeni_veri["Group"]

In [38]:
yeni_veri.drop("Group",axis=1,inplace=True)

In [39]:
# datayı görelim

yeni_veri.describe()

,M/F,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF
count,373.000000,373.000000,373.000000,373.000000,373.000000,373.000000,373.000000,373.000000,373.000000
mean,0.428954,77.013405,14.597855,2.460452,27.342318,1.774799,1488.128686,662.611260,1098.439678
std,0.495592,7.640957,2.876339,1.104665,3.673329,2.291337,176.139286,203.835828,341.268372
min,0.000000,60.000000,6.000000,1.000000,4.000000,0.000000,1106.000000,7.000000,1.000000
25%,0.000000,71.000000,12.000000,2.000000,27.000000,0.000000,1357.000000,694.000000,1054.000000
50%,0.000000,77.000000,15.000000,2.000000,29.000000,0.000000,1470.000000,724.000000,1183.000000
75%,1.000000,82.000000,16.000000,3.000000,30.000000,5.000000,1597.000000,751.000000,1291.000000
max,1.000000,98.000000,23.000000,5.000000,30.000000,5.000000,2004.000000,837.000000,1587.000000


In [40]:
cols = yeni_veri.columns

cols

Index(['M/F', 'Age', 'EDUC', 'SES', 'MMSE', 'CDR', 'eTIV', 'nWBV', 'ASF'], dtype='object')

In [41]:
scaler = MinMaxScaler()

# scaler'ı yeni_veri üzerinde eğitelim ve 
# yeni_veriyi scale edelim
yeni_veri = scaler.fit_transform(yeni_veri)

In [42]:
type(yeni_veri)

numpy.ndarray

In [43]:
# datayı DataFrame yapalım tekrar

yeni_veri = pd.DataFrame(yeni_veri, columns=[cols])

In [44]:
yeni_veri.head()

,M/F,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF
0,1.0,0.710526,0.470588,0.250000,0.884615,0.0,0.981069,0.830120,0.556116
1,1.0,0.736842,0.470588,0.250000,1.000000,0.0,1.000000,0.812048,0.551702
2,1.0,0.394737,0.352941,0.365113,0.730769,1.0,0.636971,0.878313,0.658890
3,1.0,0.421053,0.352941,0.365113,0.923077,1.0,0.703786,0.850602,0.063052
4,1.0,0.526316,0.352941,0.365113,0.692308,1.0,0.659243,0.836145,0.651324


In [45]:
# datayı artık gerçek olarak train-test şeklinde ayırabiliriz
# test size: %30 olsun

x_train, x_test, y_train, y_test = train_test_split(yeni_veri, y, test_size=0.3, random_state=2)

In [46]:
x_train.shape

(261, 9)

In [47]:
x_test.shape

(112, 9)

In [48]:
y_train

98     1
301    2
313    2
366    1
180    2
      ..
299    1
22     2
72     1
15     1
168    2
Name: Group, Length: 261, dtype: int8

In [49]:
y_test

173    1
205    1
70     2
118    2
32     2
      ..
337    2
267    2
129    2
160    2
131    2
Name: Group, Length: 112, dtype: int8

In [50]:
fit_rf = RandomForestClassifier(random_state=42)

In [51]:
# random state için seed'i sabitle
np.random.seed(42)

# timer başlat
start = time.time()

# parametre grid'ini oluştur
param_dist = {'max_depth': [2, 3, 4],
              'n_estimators': [100, 200, 400],
              'max_features': ['auto', 'sqrt', 'log2', None],
              'criterion': ['gini', 'entropy']}

# n_jobs = 4 ile 4 paralel işlemci çalıştır
cv_rf = GridSearchCV(fit_rf, 
                     cv = 5,
                     param_grid=param_dist, 
                     n_jobs = 4)

# cv_rf fit edelim
cv_rf.fit(x_train, y_train)

# en iyi parametreleri yazdır
print('GridSearch ile En İyi Parametreler: \n', cv_rf.best_params_)

# timer'ı bitir
end = time.time()

# süreyi yazdır
print('Grid Search için geçen zaman: {0: .2f}'.format(end - start))

GridSearch ile En İyi Parametreler: 
 {'criterion': 'gini', 'max_depth': 2, 'max_features': 'sqrt', 'n_estimators': 100}
Grid Search için geçen zaman:  29.31


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
90 fits failed out of a total of 360.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
67 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py", line 1144, in wrapper
    estimator._validate_params()
  File "C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py", line 637, in _validate_params
    validate_parameter_constraints(
  File "C:\ProgramData\anaconda3\Lib\site-packages

In [52]:
# GridSearch'ten aldığımız en iyi paramtreleri modelimize verelim

fit_rf.set_params(criterion = 'gini',
                  max_features = 'sqrt', 
                  max_depth = 2,
                  n_estimators = 100)

RandomForestClassifier(max_depth=2, random_state=42)

In [53]:
# modeli train edelim
fit_rf.fit(x_train, y_train)

RandomForestClassifier(max_depth=2, random_state=42)

In [54]:
# değişkenlerin önemini kullanma
pd.concat((pd.DataFrame(x_train.columns, columns = ['feature']), 
           pd.DataFrame(fit_rf.feature_importances_, columns = ['importance'])), 
          axis = 1).sort_values(by='importance', ascending = False)

,feature,importance
5,"(CDR,)",0.487366
4,"(MMSE,)",0.315131
7,"(nWBV,)",0.071501
2,"(EDUC,)",0.038115
0,"(M/F,)",0.037314
3,"(SES,)",0.017131
6,"(eTIV,)",0.012677
8,"(ASF,)",0.012573
1,"(Age,)",0.008191


In [55]:
# test data ile tahmin yap

y_pred = fit_rf.predict(x_test)

y_pred

array([1, 1, 2, 2, 2, 2, 1, 2, 2, 1, 2, 1, 1, 2, 2, 2, 2, 1, 1, 2, 2, 2,
       1, 1, 2, 1, 1, 1, 2, 2, 2, 1, 2, 2, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1,
       1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2,
       2, 1, 2, 1, 2, 1, 2, 2, 2, 1, 1, 2, 2, 1, 2, 1, 2, 2, 2, 1, 2, 1,
       1, 1, 2, 2, 2, 2, 1, 1, 2, 1, 2, 2, 1, 2, 2, 1, 1, 1, 2, 2, 2, 2,
       2, 2], dtype=int8)

In [56]:
# tahmin kalitesini ölçme accuracy
print("Modelin Accuracy Score'u: {0:0.4f}".format(accuracy_score(y_test, y_pred)))

Modelin Accuracy Score'u: 0.9286


In [57]:
# classification report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.90      1.00      0.95        43
           2       0.95      0.98      0.97        62

    accuracy                           0.93       112
   macro avg       0.62      0.66      0.64       112
weighted avg       0.87      0.93      0.90       112



C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [58]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score, r2_score

In [59]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Doğruluk Oranı: {accuracy:.2f}")

r2 = r2_score(y_test, y_pred)
print(f"R^2 Skoru: {r2:.2f}")

mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error (MAE): {mae:.2f}")

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse:.2f}")

rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")

Doğruluk Oranı: 0.93
R^2 Skoru: 0.60
Mean Absolute Error (MAE): 0.10
Mean Squared Error (MSE): 0.15
Root Mean Squared Error (RMSE): 0.39
